# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
df = data.groupby(["CustomerID", "ProductName"])["Quantity"].sum().reset_index()
display(df.head())

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
df_pivot = pd.pivot_table(df, values="Quantity", index=["ProductName"], columns=["CustomerID"], fill_value = 0)
display(df_pivot.describe())

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
count,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,...,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000
mean,0.139381,0.170354,0.143805,0.148230,0.161504,0.146018,0.170354,0.134956,0.146018,0.148230,...,3.816372,3.595133,3.871681,3.152655,3.982301,3.761062,3.871681,3.871681,4.535398,4.037611
std,0.371427,0.410188,0.369732,0.391338,0.397359,0.377772,0.431269,0.360978,0.377772,0.396963,...,9.001325,9.392041,9.930418,8.794584,10.162958,9.691036,9.502545,10.069003,10.338231,10.072504
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000,2.000000,...,25.000000,50.000000,50.000000,50.000000,75.000000,50.000000,50.000000,50.000000,50.000000,50.000000


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
similarity = squareform(pdist(df_pivot.T, 'euclidean'))
distances = pd.DataFrame(1/(1 + similarity), index=df_pivot.columns, columns=df_pivot.columns)
display(distances.head())

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
top = distances[3909].sort_values(ascending=False)
top.head(6)

CustomerID
3909    1.000000
3535    0.087410
3074    0.087410
1072    0.086333
33      0.084626
3317    0.084626
Name: 3909, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
df5 = data.loc[ data["CustomerID"].isin(top.head(6).index[1:])]
df5.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
242,3535,Lana,Peters,2399506,270,Cheese Cloth No 100,1
527,33,Lindsay,Santana,1899667,432,Beer - Original Organic Lager,1
1017,33,Lindsay,Santana,6509275,417,"Lamb - Pieces, Diced",1
1042,3535,Lana,Peters,4503728,417,"Lamb - Pieces, Diced",1
1362,3535,Lana,Peters,2575012,159,Eggplant - Asian,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
df6 = df5.groupby("ProductName")["Quantity"].sum().sort_values(ascending=False)
display(df6.head())

ProductName
Pepper - Black, Whole           4
Soup - Campbells Bean Medley    3
Fondant - Icing                 3
Cod - Black Whole Fillet        3
Wine - Chardonnay South         3
Name: Quantity, dtype: int64

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
df7 = data.loc[data["CustomerID"]==3909].merge(df6, how="outer", left_on="ProductName", right_on="ProductName")

df7bis = df7.loc[ ~(df7.ProductName.isin(data.loc[data["CustomerID"]==3909]["ProductName"]))]
print(df7.shape, df7bis.shape,data.loc[data["CustomerID"]==3909].shape)
display(df7bis.sort_values(by=['Quantity_y'], ascending=False)["ProductName"][:5])


(246, 8) (184, 8) (62, 7)


62       Pepper - Black, Whole
64    Cod - Black Whole Fillet
65     Wine - Chardonnay South
66                Sherry - Dry
67     Soupfoamcont12oz 112con
Name: ProductName, dtype: object

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [11]:

ids = pd.unique(data["CustomerID"])
recom = []

def create_recommendations(c_id, distances):
    tops = distances[c_id].sort_values(ascending=False)
    step5 = data.loc[ data["CustomerID"].isin(tops.head(6).index[1:])]
    step6 = step5.groupby("ProductName")["Quantity"].sum().sort_values(ascending=False)
    step7 = data.loc[data["CustomerID"]==c_id].merge(step6, how="outer", left_on="ProductName", \
                                                     right_on="ProductName")
    step7bis = step7.loc[ ~(step7.ProductName.isin(data.loc[data["CustomerID"]==c_id]["ProductName"]))]
    return list(step7bis.sort_values(by=['Quantity_y'], ascending=False)["ProductName"][:5])

for e in ids:
    recom.append(create_recommendations(e, distances))
    

dic = {
    "CustomerID": ids,
    "Recommendations": recom
}

print(dic)

{'CustomerID': array([61288, 77352, 40094, 23548, 78981, 83106, 11253, 35107, 15088,
       26031, 86595, 46715, 91013, 69931, 19753,  9546, 10224, 72846,
       28976,  3267, 19260, 76530, 30196, 48818, 15166, 11443, 79060,
       82597, 62411, 42611, 45420, 83495, 47122, 30815, 53198, 18591,
       62034,  1428,  2187, 55162, 22345, 81876, 48641, 81034, 29681,
       87947, 28867, 69967, 94420, 83158, 84364, 44742,  7772, 47303,
       75791, 77756, 97324, 13912, 52700, 21734,  6817, 21095, 28875,
       89588,  1034, 54827, 29287, 11102, 52155, 50748, 88552, 80938,
       60481, 92168, 66232, 64348, 37510, 44408, 89254,   477, 51483,
       97201, 42157, 52091, 86065, 18027, 93882, 42303, 32010, 53997,
       63033, 76475, 98185, 84779, 82146, 54495,  3903, 79854, 25449,
       85496, 95017, 33759, 43652, 35873, 97769,   891, 85161, 25327,
       25624, 68416, 44090, 71595, 68336, 38497, 10016, 36751, 58421,
       45498, 50774, 79997, 45313, 55715, 40901,  1577,  3909, 18480,
     

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [12]:
df9 = pd.DataFrame(dic)

df9bis = pd.concat([df9["CustomerID"], df9["Recommendations"].apply(pd.Series)], axis = 1)\
.rename(columns={0: "Recomendation 1", 1: "Recomendation 2", 2: "Recomendation 3", 3: "Recomendation 4",\
                4: "Recomendation 5",})

display(df9bis.head())

,CustomerID,Recomendation 1,Recomendation 2,Recomendation 3,Recomendation 4,Recomendation 5
0,61288,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,Macaroons - Two Bite Choc,"Water - Mineral, Natural"
1,77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
2,40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Tuna - Salad Premix,"Oregano - Dry, Rubbed",Halibut - Steaks
3,23548,Squid - Tubes / Tenticles 10/20,Wanton Wrap,Pernod,Puree - Mocha,Flavouring - Orange
4,78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Beef - Inside Round


In [13]:
df9bis.loc[df9bis["CustomerID"]==200]

,CustomerID,Recomendation 1,Recomendation 2,Recomendation 3,Recomendation 4,Recomendation 5
391,200,Soup - Campbells Bean Medley,Bay Leaf,Pork - Kidney,Wanton Wrap,Muffin - Carrot Individual Wrap


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [14]:
similarity2 = squareform(pdist(df_pivot.T, 'cosine'))
distances2 = pd.DataFrame(1/(1 + similarity2), index=df_pivot.columns, columns=df_pivot.columns)

ids2 = pd.unique(data["CustomerID"])
recom2 = []

for e in ids2:
    recom2.append(create_recommendations(e, distances2))
    

dic2 = {
    "CustomerID": ids2,
    "Recommendations": recom2
}

df10 = pd.DataFrame(dic2)
df10bis = pd.concat([df10["CustomerID"], df10["Recommendations"].apply(pd.Series)], axis = 1)\
.rename(columns={0: "Recomendation 1", 1: "Recomendation 2", 2: "Recomendation 3", 3: "Recomendation 4",\
                4: "Recomendation 5",})

display(df10bis.head())

,CustomerID,Recomendation 1,Recomendation 2,Recomendation 3,Recomendation 4,Recomendation 5
0,61288,Jagermeister,Wine - Two Oceans Cabernet,Tofu - Firm,Appetizer - Sausage Rolls,Dc - Frozen Momji
1,77352,Bread - Rye,Fondant - Icing,Lettuce - Frisee,Bread - French Baquette,Pork - Kidney
2,40094,Juice - V8 Splash,Beef - Inside Round,"Bread - Roll, Soft White Round",Truffle Cups - Brown,Cake - Mini Cheesecake
3,23548,Juice - Orange,Bagel - Plain,Veal - Sweetbread,Olives - Stuffed,Cinnamon Buns Sticky
4,78981,Wine - Blue Nun Qualitatswein,Cookies - Assorted,Cheese - Cottage Cheese,Baking Powder,Quiche Assorted


In [15]:
df10bis.loc[df10bis["CustomerID"]==200]

,CustomerID,Recomendation 1,Recomendation 2,Recomendation 3,Recomendation 4,Recomendation 5
391,200,Snapple Lemon Tea,Longos - Grilled Salmon With Bbq,General Purpose Trigger,"Thyme - Lemon, Fresh",Tomatoes Tear Drop
